<a href="https://colab.research.google.com/github/jhbell2015/GDL_code/blob/master/03_01_autoencoder_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오토인코더

In [1]:
from google.colab import drive
import os

if os.path.exists('/content/drive') :
    drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os

GDL = '/content/drive/MyDrive/GDL_code'

sys.path.append(GDL)

In [3]:
import os

from utils.loaders import load_mnist
from models.AE import Autoencoder

## 매개변수 설정

In [4]:
# 실행 매개변수
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])
RUN_FOLDER = os.path.join(GDL, RUN_FOLDER)

[SECTION, RUN_ID, DATA_NAME, RUN_FOLDER]

['vae',
 '0001',
 'digits',
 '/content/drive/MyDrive/GDL_code/run/vae/0001_digits']

In [5]:
if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER) # create deep subdirectory at once
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

## 데이터 적재

In [6]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## 신경망 구조 정의

In [7]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if MODE == 'build':
    AE.save(RUN_FOLDER)
else:
    AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, HTML
import io

# Example models
model1 = Sequential([Dense(32, input_shape=(100,), activation='relu')])
model2 = Sequential([Dense(64, input_shape=(100,), activation='relu')])

# Function to capture summary as string
def get_summary(model):
    stream = io.StringIO()
    model.summary(print_fn=lambda x: stream.write(x + "\n"))
    return stream.getvalue()

# Get summaries
summary1 = get_summary(AE.encoder)
summary2 = get_summary(AE.decoder)

# Display side by side in HTML
display(HTML(f"""
<table>
<tr style="vertical-align:top;">
<td style="background:rgb(0,60,0)"><div style="color:rgb(255,100,100)">encoder :</div><pre style="font-family:Consolas; font-size:10px;">{summary1}</pre></td>
<td style="background:rgb(0,60,60)"><div style="color:rgb(255,100,100)">encoder :</div><pre style="font-family:Consolas; font-size:10px;">{summary2}</pre></td>
</tr>
</table>
"""))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


"encoder :Model: ""functional"" ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓ ┃ Layer (type) ┃ Output Shape ┃ Param # ┃ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩ │ encoder_input (InputLayer) │ (None, 28, 28, 1) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ encoder_conv_0 (Conv2D) │ (None, 28, 28, 32) │ 320 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu (LeakyReLU) │ (None, 28, 28, 32) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ encoder_conv_1 (Conv2D) │ (None, 14, 14, 64) │ 18,496 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu_1 (LeakyReLU) │ (None, 14, 14, 64) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ encoder_conv_2 (Conv2D) │ (None, 7, 7, 64) │ 36,928 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu_2 (LeakyReLU) │ (None, 7, 7, 64) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ encoder_conv_3 (Conv2D) │ (None, 7, 7, 64) │ 36,928 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu_3 (LeakyReLU) │ (None, 7, 7, 64) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ flatten (Flatten) │ (None, 3136) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ encoder_output (Dense) │ (None, 2) │ 6,274 │ └─────────────────────────────────┴────────────────────────┴───────────────┘ Total params: 98,946 (386.51 KB) Trainable params: 98,946 (386.51 KB) Non-trainable params: 0 (0.00 B)","encoder :Model: ""functional_1"" ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓ ┃ Layer (type) ┃ Output Shape ┃ Param # ┃ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩ │ decoder_input (InputLayer) │ (None, 2) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ dense (Dense) │ (None, 3136) │ 9,408 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ reshape (Reshape) │ (None, 7, 7, 64) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ decoder_conv_t_0 │ (None, 7, 7, 64) │ 36,928 │ │ (Conv2DTranspose) │ │ │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu_4 (LeakyReLU) │ (None, 7, 7, 64) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ decoder_conv_t_1 │ (None, 14, 14, 64) │ 36,928 │ │ (Conv2DTranspose) │ │ │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu_5 (LeakyReLU) │ (None, 14, 14, 64) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ decoder_conv_t_2 │ (None, 28, 28, 32) │ 18,464 │ │ (Conv2DTranspose) │ │ │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ leaky_re_lu_6 (LeakyReLU) │ (None, 28, 28, 32) │ 0 │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ decoder_conv_t_3 │ (None, 28, 28, 1) │ 289 │ │ (Conv2DTranspose) │ │ │ ├─────────────────────────────────┼────────────────────────┼───────────────┤ │ activation (Activation) │ (None, 28, 28, 1) │ 0 │ └─────────────────────────────────┴────────────────────────┴───────────────┘ Total params: 102,017 (398.50 KB) Trainable params: 102,017 (398.50 KB) Non-trainable params: 0 (0.00 B)"


In [9]:
if 0:
    AE.encoder.summary()

In [10]:
if 0:
    AE.decoder.summary()

## 오토인코더 훈련

In [11]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [12]:
AE.compile(LEARNING_RATE)

In [14]:
from IPython.display import Javascript

AE.train(
    x_train[:1000]
    , batch_size = BATCH_SIZE
    , epochs = 200
    , run_folder = RUN_FOLDER
    , initial_epoch = INITIAL_EPOCH
)

Javascript('alert("train finished!")')

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.1216
Epoch 1: saving model to /content/drive/MyDrive/GDL_code/run/vae/0001_digits/weights/autoencoder.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.1205 - learning_rate: 5.0000e-04
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.0746
Epoch 2: saving model to /content/drive/MyDrive/GDL_code/run/vae/0001_digits/weights/autoencoder.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - loss: 0.0743 - learning_rate: 5.0000e-04
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0638
Epoch 3: saving model to /content/drive/MyDrive/GDL_code/run/vae/0001_digits/weights/autoencoder.weights.h5


KeyboardInterrupt: 